# Introduction to SymPDE

[SymPDE](https://github.com/pyccel/sympde) is a Symbolic Library for Partial Differential Equations and more precisely for variational problems. You can install it using
```shell
> pip3 install sympde
```

**SymPDE** presents different topological concepts that are useful when you are dealing with Partial Differential Equations. One of the main novelties is allowing semantic capturing of mathematical expressions, and hence reducing bugs. **SymPDE** can be viewed as a static compiler for PDEs, but rather than generating a decorated AST, it provides an AST that you can later on decorate.

The important notions in **SymPDE** are:

* Geometry
* Function space
* Expressions, like Linear and Bilinear Forms
* Equation

## 1. Geometry

The concept of a geometry is not really defined as in a standard library. it is more a mathematical concept, up to a homeomorphism.

**SymPDE** provides some basic geometries.
Typical imports are the following ones:

In [1]:
from sympde.topology import Line
from sympde.topology import Square
from sympde.topology import Cube
from sympde.topology import Domain

### Line
A line is defined by its **bounds**.


In [2]:
# create the interval [-1, 1]
domain = Line(bounds=[-1,1])

### Square
A square is defined by its **bounds** for each **axis**.

In [3]:
# create the square [-1, 1] x [-1, 1]
domain = Square(bounds1=[-1,1], bounds2=[-1,1])

### Cube
A cube is defined by its **bounds** for each **axis**.

In [4]:
# create the cube [-1, 1] x [-1, 1] x [-1, 1]
domain = Cube(bounds1=[-1,1], bounds2=[-1,1], bounds3=[-1,1])

### Domain
Represents an undefined domain.
A domain is defined by at least one interior domain and possible boundaries.
A domain without a boundary is either infinite or periodic.
A domain can also be constructed from a connectivity, in which case, only the
name and connectivity need to be passed.

This concept can be used to create a more complicated *topological* geometries.

In [92]:
from sympde.topology import Domain
from sympde.topology import InteriorDomain
from sympde.topology import Union
from sympde.topology import Interface
from sympde.topology import Connectivity
from sympde.topology import Boundary

In [93]:
# ... create a domain with 2 subdomains A and B
A = InteriorDomain('A', dim=3)
B = InteriorDomain('B', dim=3)

connectivity = Connectivity()

bnd_A_1 = Boundary('Gamma_1', A)
bnd_A_2 = Boundary('Gamma_2', A)
bnd_A_3 = Boundary('Gamma_3', A)

bnd_B_1 = Boundary('Gamma_1', B)
bnd_B_2 = Boundary('Gamma_2', B)
bnd_B_3 = Boundary('Gamma_3', B)

connectivity['I'] = Interface('I', bnd_A_1, bnd_B_2)

Omega = Domain('Omega',
               interiors=[A, B],
               boundaries=[bnd_A_2, bnd_A_3, bnd_B_1, bnd_B_3],
               connectivity=connectivity)

You can then export this topology into a **hdf5** file:

In [94]:
# export
Omega.export('omega.h5')

And read it using:

In [95]:
domain = Domain.from_file('omega.h5')

### PeriodicDomain
**TODO**

## 2. Function Spaces

Once you have a topological domain, you can create a function space over it. There are two kinds of function spaces:
    
* Scalar function space
* Vector function space

In [96]:
from sympde.topology import ScalarFunctionSpace
from sympde.topology import VectorFunctionSpace

### Scalar Function space

In [140]:
# create a generic domain in 3D
domain = Cube()

# create a scalar function space in 3D
V = ScalarFunctionSpace('V', domain)

Scalar function spaces can be typed also. This can be achieved by specifying the keyword **kind**

In this example, we create the Sobolev space $H^1(\Omega)$

In [141]:
H1 = ScalarFunctionSpace('H1', domain, kind="H1")

In this example, we create the space $L^2(\Omega)$

In [142]:
L2 = ScalarFunctionSpace('L2', domain, kind="L2")

### Vector Function space

In [143]:
# create a scalar function space in 3D
V = VectorFunctionSpace('V', domain)

Vector function spaces can also be typed

In this example, we create the Sobolev space $H(\mbox{curl}, \Omega)$

In [144]:
Hcurl = VectorFunctionSpace('Hcurl', domain, kind="Hcurl")

In this example, we create the Sobolev space $H(\mbox{div}, \Omega)$

In [145]:
Hdiv = VectorFunctionSpace('Hdiv', domain, kind="Hdiv")

### Elements of Function Spaces

You can create an element of a function space, by calling the function **element_of** as in the following examples:


In [146]:
from sympde.topology import element_of

In [147]:
u0 = element_of(H1, name='u0')
u1 = element_of(Hcurl, name="u1")
u2 = element_of(Hdiv, name="u2")
u3 = element_of(L2, name="u3")

### Product of Function Spaces

You can create a product between function spaces, either by calling the constructor **ProductSpace** or simply by using the operator $*$

In [148]:
from sympde.topology import ProductSpace

In [149]:
W = ProductSpace(H1,Hcurl,Hdiv,L2)

In [150]:
W = H1 * Hcurl * Hdiv * L2

In [151]:
u0,u1,u2,u3 = element_of(W, name='u0,u1,u2,u3')

## 3. Bilinear Forms

Once you create a function space, and have the ability to create elements in it, you can start doing some funny things, like creating a bilinear form.

A **BilinearForm** is a type, but **SymPDE** uses the Curry-Howard correspondance, and defines it as a **proposition**. If the construction is a success, this means that indeed your bilinear form is bilinear with respect to its arguments, otherwise, **SymPDE** will tell you why your expression is not bilinear.

In [152]:
v = element_of(H1, name='v')
u = element_of(H1, name='u')

In [153]:
from sympde.calculus import grad, div, dot

In [154]:
from sympde.expr.expr import integral
from sympde.expr.expr import BilinearForm

In [155]:
a = BilinearForm((v,u), integral(domain, dot(grad(v), grad(u))))

The following expression is not bilineaar with respect to $u$, and **SymPDE** is able to tell you this

In [156]:
BilinearForm((v,u), integral(domain, dot(grad(v), grad(u**2))))

Failed to assert addition property
Integral(Cube, 2*l_muah*Dot(Grad(l_muah), Grad(v)) + 2*l_muah*Dot(Grad(r_muah), Grad(v)) + 2*r_muah*Dot(Grad(l_muah), Grad(v)) + 2*r_muah*Dot(Grad(r_muah), Grad(v))) != Integral(Cube, 2*l_muah*Dot(Grad(l_muah), Grad(v)) + 2*r_muah*Dot(Grad(r_muah), Grad(v)))


UnconsistentLinearExpressionError:  Expression is not linear w.r.t test functions (u,)

### Printing

You can print your bilinear form using *Latex* printer as follows

In [157]:
from IPython.display import Math
from sympde.printing.latex import latex

In [158]:
Math(latex(a))

<IPython.core.display.Math object>

### Free-variables

Elements of functions spaces that are neither test or trial functions for the bilinear form, are considered as **free-variables**.

In [185]:
psi = element_of(H1, name='psi')
a1 = BilinearForm((v,u), integral(domain, (1+psi**2)*dot(grad(v), grad(u))))

You can then call the bilinear form and specify the free variable $\psi$ as follows

In [190]:
expr = a1(v,u,psi=u)

**TODO**

there is a problem with the printing of **Integral**

## 4. Linear Forms

In [159]:
from sympde.expr.expr import LinearForm

In [160]:
from sympde.calculus import bracket

In [161]:
psi = element_of(H1, name='psi')
l = LinearForm(v, integral(domain, v*psi**2))

In [162]:
Math(latex(l))

<IPython.core.display.Math object>

### Linearization of a non linear expression

Since **SymPDE** is able to do computations, you can linearize a non linear expression.
let's take the previous example, which is non-linear w.r.t $\psi$. In order to linearize $l$, we need to tell the function **linearize** around wich function we will do the linearization, and what is the name of the perturbation, as in the following example:

In [163]:
from sympde.expr.expr import linearize

In [164]:
dpsi = element_of(H1, name='delta_psi')
b = linearize(l, psi, trials=dpsi)

If we check the type of $b$, we will see that it is a **BilinearForm**

In [165]:
type(b)

sympde.expr.expr.BilinearForm

In [166]:
Math(latex(b))

<IPython.core.display.Math object>

### Free-variables

**TODO**

## 5. Equation

Here, we consider variational problems that can be writtent as
$$
\mbox{Find}~(u_1,u_2,\ldots,u_r) \in V_1 \times V_2 \times \ldots \times V_r,~ \mbox{such that:} \\
a((v_1,\ldots, v_r), (u_1, \ldots,u_r)) = l(v_1,\ldots,v_r),\quad \forall (v_1,\ldots, v_r) \in V_1 \times V_2 \times \ldots \times V_r
$$

In [167]:
from sympde.expr import find

In [168]:
equation = find(u, forall=v, lhs=a(u, v), rhs=l(v))

In [169]:
Math(latex(equation))

<IPython.core.display.Math object>

### Essential Boundary Conditions

You can tell **SymPDE** what are the essential boundary conditions for your problem.

In [170]:
from sympde.expr import EssentialBC

In [171]:
bnd = domain.boundary

In [172]:
type(bnd)

sympde.topology.basic.Union

In [173]:
bnd

Union(Cube_\Gamma_1, Cube_\Gamma_2, Cube_\Gamma_3, Cube_\Gamma_4, Cube_\Gamma_5, Cube_\Gamma_6)

In order to set $u$ to $0$ on the whole boundary of our domain, we simply write

In [174]:
bc = EssentialBC(u, 0, bnd)

then, we add the boundary condition to our **find** statement as the following

In [175]:
equation = find(u, forall=v, lhs=a(u, v), rhs=l(v), bc=bc)

In [176]:
Math(latex(equation))

<IPython.core.display.Math object>